<a href="https://colab.research.google.com/github/shehadeh7/APS360-Project/blob/main/1D_Conv/data_processing_from_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pydub
! pip install noisereduce
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset from Kaggle (direct link: https://www.kaggle.com/dmitrybabko/speech-emotion-recognition-en)
!kaggle datasets download dmitrybabko/speech-emotion-recognition-en

100% 987M/987M [00:11<00:00, 75.7MB/s]
100% 987M/987M [00:11<00:00, 92.1MB/s]


In [ ]:
!mkdir ./data

In [ ]:
%%capture
!unzip speech-emotion-recognition-en.zip -d ./data

In [ ]:
import numpy as np
import os
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle

In [ ]:
emotions = {
  'happy': 0, 
  'sad': 1, 
  'angry': 2, 
  'disgust': 3, 
  'fear': 4, 
  'neutral': 5
}

ravdess_emotions = {
    '01': emotions['neutral'],
    '03': emotions['happy'],
    '04': emotions['sad'],
    '05': emotions['angry'],
    '06': emotions['fear'],
    '07': emotions['disgust'],
}

crema_emotions = {
  'SAD': emotions['sad'], 
  'ANG': emotions['angry'], 
  'DIS': emotions['disgust'],
  'FEA': emotions['fear'], 
  'HAP': emotions['happy'], 
  'NEU': emotions['neutral']
}

savee_emotions = {
    'a': emotions['angry'],
    'd': emotions['disgust'],
    'f': emotions['fear'],
    'h': emotions['happy'],
    'n': emotions['neutral']
}

processed_data = []

data_path = "/content/data"
for root, dirs, files in os.walk(data_path):
  for file in files:
    file_path = os.path.join(root, file)

    emotion_label = None

    dataset = file_path.split('/')[3]
    if dataset == 'Ravdess':
      emotion_label = ravdess_emotions.get(file.split('-')[2], None)
    # elif dataset == 'Crema':
    #   emotion_label = crema_emotions.get(file.split('_')[2], None)
    elif dataset == 'Tess':
      emotion_label = emotions.get(file.split('_')[2].split('.')[0], None)
    elif dataset == 'Savee':
      emotion_code = file.split('_')[1][:2]
      if (emotion_code == 'sa'):
        emotion_label = emotions['sad']
      else:
        emotion_label = savee_emotions.get(emotion_code[0], None)

    if (emotion_label != None):
      processed_data.append([file_path, dataset, emotion_label])

In [ ]:
import librosa

def extract_feature(file_name): 

    y, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

    # Don't process corrupted audio signals
    if not np.any(y):
        return None

    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sample_rate,n_mfcc=40).T, axis=0)

    return mfccs

In [ ]:
# Obtain features from each wav file
labels = []
features = []

i = 0 # partially save results?
for data in processed_data:
    i += 1
    result = extract_feature(data[0])
    if result is not None:
        labels.append(data[2])
        features.append(result)
    if i%100 == 0:
        print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800


In [ ]:
# Create np arrays for data and labels
X = np.array(features)
Y = np.array(labels)

In [ ]:
# shuffle X and Y same way
# random_state to seed the shuffle
# X, Y = shuffle(X, Y, random_state=0)
print(X.shape)
print(Y.shape)

(3876, 40)
(3876,)


In [ ]:
#numpy save x and y
#zip together before saving?
x_path = './x_data'
y_path = './y_data'
np.save(x_path, X)
np.save(y_path, Y)

In [ ]:
! zip github_fork_data.zip x_data.npy y_data.npy

  adding: x_data.npy (deflated 7%)
  adding: y_data.npy (deflated 96%)
